<a href="https://colab.research.google.com/github/soul3510/Stock-Analyzer/blob/main/Eyal_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [253]:
# Install necessary packages
!pip install gradio yfinance requests lxml beautifulsoup4 pandas

import yfinance as yf
import requests
from lxml import html
import gradio as gr
import pandas as pd
import random

def load_nasdaq_tickers():
    """
    Loads NASDAQ-listed ticker symbols from the provided FTP URL.
    Excludes any non-ticker entries such as headers or footers.
    """
    nasdaq_url = "ftp://ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt"
    try:
        nasdaq_df = pd.read_csv(nasdaq_url, sep='|')
        nasdaq_tickers = nasdaq_df['Symbol'].tolist()
        # Remove the last element if it's not a ticker symbol
        if nasdaq_tickers[-1] == 'File Creation Time':
            nasdaq_tickers = nasdaq_tickers[:-1]
        return nasdaq_tickers
    except Exception as e:
        print(f"Error loading NASDAQ tickers: {e}")
        return []

# Load NASDAQ tickers once when the script runs
nasdaq_tickers = load_nasdaq_tickers()

def color_text(text, color):
    """
    Returns HTML-formatted text with the specified color.
    """
    return f"<span style='color:{color}'>{text}</span>"

def run_analysis(ticker_symbol):
    """
    Analyzes the given ticker symbol based on predefined financial criteria.
    Returns a Markdown-formatted string with the analysis results.
    """
    results = []

    try:
        ticker = yf.Ticker(ticker_symbol)
        info = ticker.info
    except Exception as e:
        return f"**Error fetching data for {ticker_symbol}:** {e}"

    results.append(f"**🔍 Checking:** {ticker_symbol}\n")

    current_price = info.get("currentPrice", None)
    results.append(f"**💰 Current Price:** ${current_price}\n")

    # Define criteria
    criteria = [
        {
            "name": "Enterprise Value (EV) / EBITDA",
            "key": "ebitda",  # We'll compute EV/EBITDA using this key and enterpriseValue
            "threshold": 10.0,
            "comparison": "less",
            "points": 10,
            "expected_str": "<10"
        },
        {
            "name": "Price To Sales",
            "key": "priceToSalesTrailing12Months",
            "threshold": 2.0,
            "comparison": "less",
            "points": 10,
            "expected_str": "<2"
        },
        {
            "name": "Profit Margin",
            "key": "profitMargins",
            "threshold": 0.10,
            "comparison": "greater",
            "points": 10,
            "expected_str": ">10%"
        },
        {
            "name": "Operating Margin",
            "key": "operatingMargins",
            "threshold": 0.15,
            "comparison": "greater",
            "points": 10,
            "expected_str": ">15%"
        },
        {
            "name": "Return on Equity",
            "key": "returnOnEquity",
            "threshold": 0.15,
            "comparison": "greater",
            "points": 10,
            "expected_str": ">15%"
        },
        {
            "name": "PEG Ratio",
            "key": "pegRatio",
            "threshold": 1.5,
            "comparison": "less",
            "points": 10,
            "expected_str": "<1.5"
        },
        {
            "name": "P/E Ratio",
            "key": "trailingPE",
            "threshold": 20,
            "comparison": "less",
            "points": 10,
            "expected_str": "<20"
        },
        {
            "name": "Debt/Equity",
            "key": "debtToEquity",
            "threshold": 100,
            "comparison": "less",
            "points": 10,
            "expected_str": "<100%"
        },
        {
            "name": "Current Ratio",
            "key": "currentRatio",
            "threshold": 1.5,
            "comparison": "greater",
            "points": 5,
            "expected_str": ">1.5"
        },
        {
            "name": "Levered Free Cash Flow",
            "key": "leveredFreeCashFlow",
            "threshold": 0,
            "comparison": "greater",
            "points": 10,
            "expected_str": ">0"
        },
        {
            "name": "Dividend Yield",
            "key": "dividendYield",
            "threshold": 0.02,
            "comparison": "greater",
            "points": 5,
            "expected_str": ">2%"
        },
        {
            "name": "Revenue Growth",
            "key": "revenueGrowth",
            "threshold": 0.05,
            "comparison": "greater",
            "points": 10,
            "expected_str": ">5%"
        },
        {
            "name": "Market Cap",
            "key": "marketCap",
            "threshold": 0,
            "comparison": "greater",
            "points": 0,
            "expected_str": ""
        }
    ]

    # Initialize scoring
    total_points = 0
    max_points = sum(c["points"] for c in criteria)

    for c in criteria:
        if c["name"] == "PEG Ratio":
            # Special handling for PEG Ratio
            try:
                url = f"https://finance.yahoo.com/quote/{ticker_symbol}"
                response = requests.get(url)
                tree = html.fromstring(response.text)

                peg_xpath = '//div[contains(@class,"container")]//li[p[contains(text(),"PEG Ratio")]]/p[2]/text()'
                peg_elements = tree.xpath(peg_xpath)

                if peg_elements:
                    peg_ratio_text = peg_elements[0].strip()
                    value = float(peg_ratio_text)
                else:
                    value = None
            except Exception as e:
                value = None

            if value is not None:
                meets_criteria = (value < c["threshold"]) if c["comparison"] == "less" else (value > c["threshold"])
                display_value = f"{value:.2f}"
                if meets_criteria:
                    total_points += c["points"]
                    status = color_text("**Very Good**", "green")
                else:
                    status = color_text("**Not Good Enough**", "red")
                results.append(f"**{c['name']}:** {display_value} --> Expected: {c['expected_str']} --> {status}")
            else:
                results.append(f"**{c['name']}:** Data not available")

        elif c["name"] == "Enterprise Value (EV) / EBITDA":
            # Special handling for Enterprise Value/EBITDA
            try:
                ev = info.get("enterpriseValue", None)
                ebitda = info.get("ebitda", None)
                if ev is not None and ebitda is not None and ebitda != 0:
                    ev_ebitda = ev / ebitda
                else:
                    ev_ebitda = None
            except Exception as e:
                ev_ebitda = None

            if ev_ebitda is not None:
                meets_criteria = (ev_ebitda < c["threshold"]) if c["comparison"] == "less" else (ev_ebitda > c["threshold"])
                display_value = f"{ev_ebitda:.2f}"
                if meets_criteria:
                    total_points += c["points"]
                    status = color_text("**Very Good**", "green")
                else:
                    status = color_text("**Not Good Enough**", "red")
                results.append(f"**{c['name']}:** {display_value} --> Expected: {c['expected_str']} --> {status}")
            else:
                results.append(f"**{c['name']}:** Data not available")

        elif c["name"] == "Market Cap":
            # Special handling for Market Cap
            try:
                url = f"https://finance.yahoo.com/quote/{ticker_symbol}"
                response = requests.get(url)
                tree = html.fromstring(response.text)

                market_cap_xpath = '//div[contains(@class,"container")]//li[p[contains(text(),"Market Cap")]]/p[2]/text()'
                market_cap_elements = tree.xpath(market_cap_xpath)

                if market_cap_elements:
                    market_cap_text = market_cap_elements[0].strip()

                    def parse_market_cap(s):
                        s = s.upper().replace(',', '')
                        if s.endswith('B'):
                            return float(s[:-1]) * 10**9
                        elif s.endswith('M'):
                            return float(s[:-1]) * 10**6
                        elif s.endswith('K'):
                            return float(s[:-1]) * 10**3
                        else:
                            return float(s)

                    value = parse_market_cap(market_cap_text)
                else:
                    value = None
            except Exception as e:
                value = None

            if value is not None:
                # Classify the market cap
                if value > 10_000_000_000:
                    classification = "Very Good (Large-Cap)"
                elif value > 2_000_000_000:
                    classification = "Good (Mid-Cap)"
                elif value > 300_000_000:
                    classification = "Moderate (Small-Cap)"
                else:
                    classification = "High Risk (Micro-Cap)"
                results.append(f"**{c['name']}:** ${value:,.2f} --> {classification}")
            else:
                results.append(f"**{c['name']}:** Data not available")

        else:
            # For all other criteria, use info directly
            try:
                value = info.get(c["key"], None)
            except Exception as e:
                value = None

            if value is not None:
                meets_criteria = (value > c["threshold"]) if c["comparison"] == "greater" else (value < c["threshold"])

                if c["key"] in ["profitMargins", "operatingMargins", "returnOnEquity", "dividendYield", "revenueGrowth"]:
                    display_value = f"{value*100:.2f}%"
                elif c["key"] == "debtToEquity":
                    display_value = f"{value:.2f}%"
                else:
                    display_value = f"${value:,.2f}" if "Price" in c["name"] or "Ratio" in c["name"] else str(value)

                if meets_criteria:
                    total_points += c["points"]
                    status = color_text("**Very Good**", "green")
                else:
                    status = color_text("**Not Good Enough**", "red")
                results.append(f"**{c['name']}:** {display_value} --> Expected: {c['expected_str']} --> {status}")
            else:
                results.append(f"**{c['name']}:** Data not available")

    # Calculate final score
    final_score_percentage = (total_points / max_points) * 100

    results.append(f"\n**📊 Final Score:** {final_score_percentage:.2f}%")

    # Return all results as a single string with Markdown formatting
    return "\n".join(results)

def gradio_interface(ticker):
    """
    Interface function for analyzing a specific ticker.
    """
    return run_analysis(ticker)

def analyze_random_ticker():
    """
    Selects a random ticker from the NASDAQ list and runs the analysis.
    """
    if not nasdaq_tickers:
        return "NASDAQ ticker list is empty or failed to load."
    random_ticker = random.choice(nasdaq_tickers)
    return run_analysis(random_ticker)

# Set up a Gradio interface using Blocks for more flexibility
with gr.Blocks() as demo:
    gr.Markdown("# 📈 Stock Analyzer (By Eyal Sooliman)")
    gr.Markdown("""
    Enter a stock ticker symbol to analyze its financial health based on various metrics, or click the button below to analyze a random NASDAQ ticker.
    """)

    with gr.Row():
        ticker_input = gr.Textbox(
            label="Enter Stock Ticker (e.g., AAPL, MSFT)",
            placeholder="AAPL"
        )
        analyze_button = gr.Button("Analyze Ticker")

    analyze_random_button = gr.Button("Analyze Random NASDAQ Ticker")

    output = gr.Markdown()

    # Connect the buttons to their respective functions
    analyze_button.click(fn=gradio_interface, inputs=ticker_input, outputs=output)
    analyze_random_button.click(fn=analyze_random_ticker, inputs=None, outputs=output)

# Launch the Gradio interface
demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8c9cbdbd9ecd54c65b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
